## 准备数据

In [20]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [21]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [34]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(tf.random.normal([784, 256], stddev=0.1))# 784=28*28
        self.b1 = tf.Variable(tf.zeros([256]))
        self.W2 = tf.Variable(tf.random.normal([256, 10], stddev=0.1))
        self.b2 = tf.Variable(tf.zeros([10]))
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        x = tf.reshape(x, [-1, 784])
        h1 = x @ self.W1 + self.b1
        h1 = tf.nn.relu(h1)
        logits = h1 @ self.W2 + self.b2
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [35]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [36]:
train_data, test_data = mnist_dataset()
for epoch in range(500):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.6763237 ; accuracy 0.099366665
epoch 1 : loss 2.6253285 ; accuracy 0.10226667
epoch 2 : loss 2.5790157 ; accuracy 0.105283335
epoch 3 : loss 2.5366714 ; accuracy 0.11045
epoch 4 : loss 2.4977074 ; accuracy 0.11686666
epoch 5 : loss 2.4616456 ; accuracy 0.1223
epoch 6 : loss 2.428096 ; accuracy 0.12883334
epoch 7 : loss 2.3967302 ; accuracy 0.13688333
epoch 8 : loss 2.367275 ; accuracy 0.14513333
epoch 9 : loss 2.3394988 ; accuracy 0.1549
epoch 10 : loss 2.3132033 ; accuracy 0.16626666
epoch 11 : loss 2.2882214 ; accuracy 0.17661667
epoch 12 : loss 2.2644062 ; accuracy 0.18763334
epoch 13 : loss 2.2416308 ; accuracy 0.19913334
epoch 14 : loss 2.2197797 ; accuracy 0.21141666
epoch 15 : loss 2.198755 ; accuracy 0.22251667
epoch 16 : loss 2.1784728 ; accuracy 0.23438333
epoch 17 : loss 2.1588578 ; accuracy 0.24603334
epoch 18 : loss 2.139845 ; accuracy 0.25806665
epoch 19 : loss 2.1213765 ; accuracy 0.26956666
epoch 20 : loss 2.1034024 ; accuracy 0.28066668
epoch 21 : loss